# Add new labeled data 🛰️

**Description:** Stand alone notebook for adding new training and evaluation data. 

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nasaharvest/openmapflow/blob/main/openmapflow/notebooks/new_data.ipynb)

# 1. Setup

If you don't already have one, obtain a Github Personal Access Token using the steps [here](https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/creating-a-personal-access-token). Save this token somewhere private.

In [ ]:
!pip install "ipywidgets>=7,<8" -q # https://github.com/googlecolab/colabtools/issues/3020

In [ ]:
from ipywidgets import Password, Text, VBox
inputs = [
      Password(description="Github Token:"),
      Text(description='Github Email:'),
      Text(description='Github User:'),
      Text(description='Github URL:'),
]
VBox(inputs)

In [ ]:
token = inputs[0].value
email = inputs[1].value
username = inputs[2].value
github_url = inputs[3].value

!git config --global user.email $username
!git config --global user.name $email
!git clone {github_url.replace("https://", f"https://{username}:{token}@")}

In [ ]:
from pathlib import Path
path_to_yaml = input("Path to openmapflow.yaml: ")
%cd {Path(path_to_yaml).parent}

In [ ]:
!pip install openmapflow[data]==0.2.2rc1 dvc[gs] -q
!pip install pyyaml==5.4.1 -q

In [ ]:
!gcloud auth application-default login

In [ ]:
from google.colab import files
from openmapflow.utils import colab_gee_gcloud_login
from openmapflow.config import PROJECT_ROOT, DataPaths, GCLOUD_PROJECT_ID
from openmapflow.raw_labels import _read_in_file

In [ ]:
colab_gee_gcloud_login(GCLOUD_PROJECT_ID)

In [ ]:
# Existing branches
!git branch -r

In [ ]:
choice = input("a) Checking progress of dataset creation OR \nb) Creating new dataset \na/b: ")
if choice == "a":
  branch_name = input("Existing branch name: ")
  !git checkout {branch_name}
  !git pull
elif choice == "b":
  branch_name = input("New branch name: ")
  !git checkout -b {branch_name}
else:
  print(f"Invalid choice: {choice}, must be 'a' or 'b'")


# 2. Download latest data
Data is stored in remote storage (ie. Google Drive) so authentication is necessary.

In [ ]:
!dvc pull -q

# 3. Upload labels

In [ ]:
dataset_name = input("Dataset name (suggested format: <Country_Region_Year>): ")
while True:
    dataset_dir = PROJECT_ROOT / DataPaths.RAW_LABELS / dataset_name
    if dataset_dir.exists() and len(list(dataset_dir.iterdir())) > 0:
        dataset_name = input("Dataset name already exists, try a different name: ")
    else:
        dataset_dir.mkdir(exist_ok=True)
        break

print("--------------------------------------------------")
print(f"Dataset: {dataset_name} directory created")
print("---------------------------------------------------")
uploaded = files.upload()
for file_name in uploaded.keys():
    Path(file_name).rename(dataset_dir / file_name)

In [ ]:
# Assess dataset
df = _read_in_file(dataset_dir / file_name)
df.head()

# 4. Create dataset
<img src="https://storage.googleapis.com/harvest-public-assets/openmapflow/new_data.png"/>

`openmapflow create-datasets` creates datasets from labels and earth observation data referenced in datasets.py.

It first checks if the necessary earth observation data is already available in Cloud Storage, or if an active Earth Engine task is already active. So Google Cloud and Earth Engine authentication is needed.

In [ ]:
user_confirmation = input(
    "Open datasets.py and add a `LabeledDataset` object representing the labels just added.\n"+
    "Added `LabeledDataset y/[n]: "
)
if user_confirmation.lower() != "y":
    print("New features can only be created when a `LabeledDataset` object is added.")

In [ ]:
import importlib
from openmapflow.labeled_dataset import create_datasets
import datasets as ds
importlib.reload(ds)
create_datasets(ds.datasets)

# 5. Push new dataset to the repository

In [ ]:
# Pushing to remote storage
!dvc commit
!dvc push

In [ ]:
# Pushing reference to github
commit_message = input("Commit message: ")
!git add .
!git commit -m '{commit_message}'
!git push --set-upstream origin {branch_name}

Create a Pull Request so the data can be merged into the main branch.